In [4]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

In [5]:
values = scipy.io.loadmat("test2.mat")

In [6]:
percent = 0.8
p = np.random.permutation(len(values['u']))
actions = values["u"][p]
states = values['x'][p]
n = int(percent*actions.shape[0])
actions_train, actions_validate, states_train, states_validate = actions[:n], actions[n:], states[n:], states[:n]

In [7]:
model = keras.Sequential([
    keras.Input(shape=(2)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1),
])
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                48        
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 337
Trainable params: 337
Non-trainable params: 0
_________________________________________________________________


In [9]:
with tf.device('/cpu:0'):
    model.compile(optimizer=keras.optimizers.Adam(), loss="mse", metrics=["mse"])
    model.fit(states, actions, epochs=10, validation_split=0.8)

Epoch 1/10
625/625 [==============================] - 5s 8ms/step - loss: 7.9556e-04 - mse: 7.9556e-04 - val_loss: 7.0923e-04 - val_mse: 7.0923e-04
Epoch 2/10
625/625 [==============================] - 4s 7ms/step - loss: 6.2305e-04 - mse: 6.2305e-04 - val_loss: 7.6125e-04 - val_mse: 7.6125e-04
Epoch 3/10
625/625 [==============================] - 5s 8ms/step - loss: 5.1774e-04 - mse: 5.1774e-04 - val_loss: 3.6371e-04 - val_mse: 3.6371e-04
Epoch 4/10
625/625 [==============================] - 5s 8ms/step - loss: 8.2126e-04 - mse: 8.2126e-04 - val_loss: 3.5669e-04 - val_mse: 3.5669e-04
Epoch 5/10
625/625 [==============================] - 5s 8ms/step - loss: 5.4132e-04 - mse: 5.4132e-04 - val_loss: 3.4651e-04 - val_mse: 3.4651e-04
Epoch 6/10
625/625 [==============================] - 5s 8ms/step - loss: 4.2262e-04 - mse: 4.2262e-04 - val_loss: 6.4207e-04 - val_mse: 6.4207e-04
Epoch 7/10
625/625 [==============================] - 5s 8ms/step - loss: 3.9559e-04 - mse: 3.9559e-04 - val_los

In [10]:
keras.models.save_model(model, filepath="./models/mpc.tf", save_format="tf")

INFO:tensorflow:Assets written to: ./models/mpc.tf\assets
